Install ortools package


In [ ]:
!pip install ortools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 16.0 MB 27.3 MB/s 


This is a solution for the Stell Mill Slab problem found on the internet.

In [ ]:
#!/usr/bin/env python3
# Copyright 2010-2022 Google LLC
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""Solves the Stell Mill Slab problem with 4 different techniques."""

# overloaded sum() clashes with pytype.
# pytype: disable=wrong-arg-types

import collections
import time

from ortools.linear_solver import pywraplp
from ortools.sat.python import cp_model

def build_problem():
    """Build problem data."""

    capacities = [1, 3, 4]
    num_colors = 5
    # num_slabs = 4
    num_slabs = 9
    # (size, color)
    orders = [
        (2, 1),
        (3, 2),
        (1, 2),
        (1, 3),
        (1, 4),
        (1, 4),
        (1, 4),
        (2, 5),
        (1, 5)
    ]

    return (num_slabs, capacities, num_colors, orders)

class SteelMillSlabSolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, orders, assign, load, loss):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__orders = orders
        self.__assign = assign
        self.__load = load
        self.__loss = loss
        self.__solution_count = 0
        self.__all_orders = range(len(orders))
        self.__all_slabs = range(len(assign[0]))
        self.__start_time = time.time()

    def on_solution_callback(self):
        """Called on each new solution."""
        current_time = time.time()
        objective = sum(self.Value(l) for l in self.__loss)
        print('Solution %i, time = %f s, objective = %i' %
              (self.__solution_count, current_time - self.__start_time,
               objective))
        self.__solution_count += 1
        orders_in_slab = [[
            o for o in self.__all_orders if self.Value(self.__assign[o][s])
        ] for s in self.__all_slabs]
        for s in self.__all_slabs:
            if orders_in_slab[s]:
                line = '  - slab %i, load = %i, loss = %i, orders = [' % (
                    s, self.Value(self.__load[s]), self.Value(self.__loss[s]))
                for o in orders_in_slab[s]:
                    line += '#%i(w%i, c%i) ' % (o, self.__orders[o][0],
                                                self.__orders[o][1])
                line += ']'
                print(line)


def steel_mill_slab(break_symmetries):
    """Solves the Steel Mill Slab Problem."""
    ### Load problem.
    (num_slabs, capacities, num_colors, orders) = build_problem()

    num_orders = len(orders)
    num_capacities = len(capacities)
    all_slabs = range(num_slabs)
    all_colors = range(num_colors)
    all_orders = range(len(orders))
    print('Solving steel mill with %i orders, %i slabs, and %i capacities' %
          (num_orders, num_slabs, num_capacities - 1))

    # Compute auxiliary data.
    widths = [x[0] for x in orders]
    colors = [x[1] for x in orders]
    max_capacity = max(capacities)
    loss_array = [
        min(x for x in capacities if x >= c) - c for c in range(max_capacity +
                                                                1)
    ]
    max_loss = max(loss_array)
    orders_per_color = [
        [o for o in all_orders if colors[o] == c + 1] for c in all_colors
    ]
    unique_color_orders = [
        o for o in all_orders if len(orders_per_color[colors[o] - 1]) == 1
    ]

    ### Model problem.

    # Create the model and the decision variables.
    model = cp_model.CpModel()
    assign = [[
        model.NewBoolVar('assign_%i_to_slab_%i' % (o, s)) for s in all_slabs
    ] for o in all_orders]
    loads = [
        model.NewIntVar(0, max_capacity, 'load_of_slab_%i' % s)
        for s in all_slabs
    ]
    color_is_in_slab = [[
        model.NewBoolVar('color_%i_in_slab_%i' % (c + 1, s)) for c in all_colors
    ] for s in all_slabs]

    # Compute load of all slabs.
    for s in all_slabs:
        model.Add(sum(assign[o][s] * widths[o] for o in all_orders) == loads[s])

    # Orders are assigned to one slab.
    for o in all_orders:
        model.AddExactlyOne(assign[o])

    # Redundant constraint (sum of loads == sum of widths).
    model.Add(sum(loads) == sum(widths))

    # Link present_colors and assign.
    for c in all_colors:
        for s in all_slabs:
            for o in orders_per_color[c]:
                model.AddImplication(assign[o][s], color_is_in_slab[s][c])
                model.AddImplication(color_is_in_slab[s][c].Not(),
                                     assign[o][s].Not())

    # At most two colors per slab.
    for s in all_slabs:
        model.Add(sum(color_is_in_slab[s]) <= 2)

    # Project previous constraint on unique_color_orders
    for s in all_slabs:
        model.Add(sum(assign[o][s] for o in unique_color_orders) <= 2)

    # Symmetry breaking.
    for s in range(num_slabs - 1):
        model.Add(loads[s] >= loads[s + 1])

    # Collect equivalent orders.
    width_to_unique_color_order = {}
    ordered_equivalent_orders = []
    for c in all_colors:
        colored_orders = orders_per_color[c]
        if not colored_orders:
            continue
        if len(colored_orders) == 1:
            o = colored_orders[0]
            w = widths[o]
            if w not in width_to_unique_color_order:
                width_to_unique_color_order[w] = [o]
            else:
                width_to_unique_color_order[w].append(o)
        else:
            local_width_to_order = {}
            for o in colored_orders:
                w = widths[o]
                if w not in local_width_to_order:
                    local_width_to_order[w] = []
                local_width_to_order[w].append(o)
            for w, os in local_width_to_order.items():
                if len(os) > 1:
                    for p in range(len(os) - 1):
                        ordered_equivalent_orders.append((os[p], os[p + 1]))
    for w, os in width_to_unique_color_order.items():
        if len(os) > 1:
            for p in range(len(os) - 1):
                ordered_equivalent_orders.append((os[p], os[p + 1]))

    # Create position variables if there are symmetries to be broken.
    if break_symmetries and ordered_equivalent_orders:
        print('  - creating %i symmetry breaking constraints' %
              len(ordered_equivalent_orders))
        positions = {}
        for p in ordered_equivalent_orders:
            if p[0] not in positions:
                positions[p[0]] = model.NewIntVar(0, num_slabs - 1,
                                                  'position_of_slab_%i' % p[0])
                model.AddMapDomain(positions[p[0]], assign[p[0]])
            if p[1] not in positions:
                positions[p[1]] = model.NewIntVar(0, num_slabs - 1,
                                                  'position_of_slab_%i' % p[1])
                model.AddMapDomain(positions[p[1]], assign[p[1]])
            # Finally add the symmetry breaking constraint.
            model.Add(positions[p[0]] <= positions[p[1]])

    # Objective.
    obj = model.NewIntVar(0, num_slabs * max_loss, 'obj')
    losses = [model.NewIntVar(0, max_loss, 'loss_%i' % s) for s in all_slabs]
    for s in all_slabs:
        model.AddElement(loads[s], loss_array, losses[s])
    model.Add(obj == sum(losses))
    model.Minimize(obj)

    ### Solve model.
    solver = cp_model.CpSolver()
    solver.parameters.num_search_workers = 8
    objective_printer = cp_model.ObjectiveSolutionPrinter()
    status = solver.Solve(model, objective_printer)

    ### Output the solution.
    if status in (cp_model.OPTIMAL, cp_model.FEASIBLE):
        print(
            'Loss = %i, time = %f s, %i conflicts' %
            (solver.ObjectiveValue(), solver.WallTime(), solver.NumConflicts()))
    else:
        print('No solution')

steel_mill_slab(False)

Solving steel mill with 9 orders, 9 slabs, and 2 capacities
Solution 0, time = 0.01 s, objective = 4
Solution 1, time = 0.02 s, objective = 2
Solution 2, time = 0.02 s, objective = 1
Loss = 1, time = 0.021229 s, 11 conflicts


In [ ]:
from ortools.linear_solver import pywraplp

def create_data_model():
    """Create the data for the example."""
    data = {}
    weights = [2, 3, 1, 1, 1, 1, 1, 2, 1]
    colors = [1, 2, 2, 3, 4, 4, 4, 5, 5]
    data['weights'] = weights
    data['colors'] = colors
    data['items'] = list(range(len(weights)))
    data['slabs'] = data['items']
    data['slab_sizes_available'] = [1, 3, 4, 0, 0, 0, 0, 0, 0]
    return data

def main():
    data = create_data_model()

    # Create the mip solver with the SCIP backend.
    solver = pywraplp.Solver.CreateSolver('SCIP')

    if not solver:
        return

    # Variables
    # x[i, j] = 1 if item i is casted in slab j.
    x = {}
    for i in data['items']:
        for j in data['slabs']:
            x[(i, j)] = solver.IntVar(0, 1, 'x_%i_%i' % (i, j))

    # y[j] = 1 if slab j is used.
    y = {}
    for j in data['slabs']:
        y[j] = solver.IntVar(0, 1, 'y[%i]' % j)

    # Constraints
    # Each item must be in exactly one slab.
    for i in data['items']:
        solver.Add(sum(x[i, j] for j in data['slabs']) == 1)

    def colorConstraints(j):
      # LINIA URMATOARE AR TREBUI SA ZICA DACA IN BIN-UL (SLAB) RESPECTIV SUNT MAI
      # PUTINE CULORI DECAT 2 SAU NU, DAR DIN PACATE NU FUNCTIONEAZA PENTRU CA
      # NU POATE APLICA SET PE x[(i, j)] * data['colors'][i] for i in data['items']
      # DAR PE data['colors'][i] for i in data['items'] DA. CE ESTE CIUDAT PENTRU
      # CA DACA FACI PRINT LA ELE ITI VOR DA INAPOI ACELAS TYPE.
      # return len(list(set(x[(i, j)] * data['colors'][i] for i in data['items']))) <= y[j] * 2

      # MOMENTAN AM PUS SA RETURNE TOTDEAUNA TRUE (ADICA CUM AR VENI O
      # CONSTRANGERE CARE NU FACE NIMIC
      return True

    # AICI ITEREAZA FIECARE BIN (SAU SLAB IN PROBLEMA NOASTA) SI TREBUIE SA
    # VERIFICE SA NU FIE MAI MULTE DE 2 CULORI IN ACELAS BIN (SAU SLAB)
    for j in data['slabs']:
        solver.Add(colorConstraints(j))

    # Each slab must contain maximum 2 different colos.
    for j in data['slabs']:
        solver.Add(
            sum(x[(i, j)] * data['weights'][i] for i in data['items']) <= y[j] *
            data['slab_sizes_available'][j])

    # Objective: minimize the number of slabs used.
    solver.Minimize(solver.Sum([y[j] for j in data['slabs']]))

    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        num_slabs = 0.
        for j in data['slabs']:
            if y[j].solution_value() == 1:
                slab_orders = []
                slab_weight = 0
                for i in data['items']:
                    if x[i, j].solution_value() > 0:
                        slab_orders.append(i)
                        slab_weight += data['weights'][i]
                if slab_weight > 0:
                    num_slabs += 1

                    print('Slab number', j + 1)

                    orders_casted = [item + 1 for item in slab_orders]

                    print('  Orders casted:', orders_casted)
                    print('  Total weight:', slab_weight)
                    print()
        print()
        print('Number of slabs used:', num_slabs)
        print('Time = ', solver.WallTime(), ' milliseconds')
    else:
        print('The problem does not have an optimal solution.')


if __name__ == '__main__':
    main()

The problem does not have an optimal solution.
